# Implement the BSTI Model

Importing Modules

In [25]:
import numpy as np
from scipy.spatial.distance import cdist
from shapely.geometry import Point, MultiPoint, Polygon
from collections import defaultdict, OrderedDict
from shapely.ops import cascaded_union
from descartes import PolygonPatch
import seaborn as sns
import re
import theano.tensor as tt
import scipy.stats
from itertools import product
import datetime, pickle as pkl, pandas as pd, pymc3 as pm
from pymc3.stats import quantiles


import isoweek
import gc

import itertools as it
import os

import pymc3 as pm, theano, theano.tensor as tt, numpy as np, pandas as pd, isoweek, pickle as pkl, datetime, time

from matplotlib import pyplot as pp

theano.config.compute_test_value = 'off' # BUG: may throw an error for flat RVs
import theano
import re, pandas as pd, datetime, numpy as np, scipy as sp, pymc3 as pm, patsy as pt, theano.tensor as tt



import time
import matplotlib
from matplotlib.collections import PatchCollection
import matplotlib.cm
from matplotlib.axes import Axes
import matplotlib.transforms as transforms
from matplotlib import pyplot as plt
from matplotlib import rc
from shapely.geometry import Polygon
from collections import OrderedDict, defaultdict
import numpy as np
from shapely.ops import cascaded_union
from descartes import PolygonPatch
import matplotlib.patheffects as PathEffects
from matplotlib.gridspec import SubplotSpec, GridSpec, GridSpecFromSubplotSpec
import pymc3 as pm
import seaborn as sns
from itertools import product
import re


Run config.py

In [2]:
diseases = ["campylobacter", "rotavirus", "borreliosis"]
prediction_regions = ["germany", "bavaria"]

combinations_age_eastwest = [(False,False),(False,True),(True,True)]
combinations = list(it.product(range(len(combinations_age_eastwest)), diseases))

Run geo_utils.py

In [3]:
R = 6365.902 #km; Radius of the earth at a location within Germany.

def jacobian_sq(latitude):
    """
        jacobian_sq(latitude)
        
    Computes the "square root" (Cholesky factor) of the Jacobian of the cartesian projection from polar coordinates (in degrees longitude, latitude) onto cartesian coordinates (in km east/west, north/south) at a given latitude (the projection's Jacobian is invariante wrt. longitude).
    """
    return R*(np.pi/180.0)*np.array([[np.abs(np.cos(np.deg2rad(latitude))), 0.0],[0.0, 1.0]])

def compute_interaction(tp_1, tp_2, σs):
    """
        compute_interaction(tp_1, tp_2, σs)
    
    For interaction effects between two counties, Gaussian basis functions with radii σs are applied to the distances between all pairs of testpoints from both counties and averaged.
    
    Arguments:
    ==========
        tp_1:       representative testpoints from within county 1
        tp_2:       representative testpoints from within county 2
        σs:         standard deviations of the Gaussian basis-functions
    """
    centroid = (tp_1.mean(axis=0) + tp_2.mean(axis=0))/2
    Σ_sq = jacobian_sq(centroid[1])
    dx = cdist(tp_1, tp_2, "mahalanobis", VI=Σ_sq**2)

    interaction = np.zeros_like(σs)
    for k,σ in enumerate(σs):
        interaction[k] = np.mean(np.exp(-0.5*(dx/σ)**2)/np.sqrt(2*np.pi*σ**2))
    return interaction
    
    
    
def compute_loss(shape, testpoints, σs, ϵ = np.random.randn(100,2)):
    """
        compute_loss(shape, testpoints, σs, ϵ)
        
    For interaction effects between two counties, Gaussian basis functions are applied to the distance between testpoints from both counties. Cases occuring outside the `shape` of the region for which the reported cases are known should similarly contribute to interaction effects, but are missing in the dataset. To compensate for this bias, the expected fraction of 'mass' that a basis function centered around a testpoint in the county would assign to points outside the region of interest is estimated through rejection sampling.
    
    Arguments:
    ==========
        
        shape:      the shape of the region of interest surrounding the county
        testpoints: representative testpoints from within the county
        σs:         standard deviations of the Gaussian basis-functions
        ϵ:          noise term used for sampling points (num_partition_samples × 2)
    """
    borderlosses = np.zeros(len(σs))
    for tp in testpoints:
        Σ_sq = jacobian_sq(tp[1])
        Σ_sqinv = np.diag(1/np.diag(Σ_sq))

        # draw points surrounding testpoint in a Gaussian with radius 1km
        other_offsets = ϵ.dot(Σ_sqinv.T)

        # check border effects for each radius
        for j,σ in enumerate(σs):
            # draw samples around each testpoint in the county and check if they land in the region
            smp = MultiPoint(tp+other_offsets*σ)
            pts = shape.intersection(smp)
            borderlosses[j] += (1 if isinstance(pts, Point) else len(pts))
    return np.clip(borderlosses/(len(testpoints)*len(ϵ)), 0.0, 1.0)


Run sampling_utils.py

In [4]:
# -*- coding: utf-8 -*-

def uniform_times_by_week(weeks, n=500):
    res = OrderedDict()
    for week in weeks:
        time_min = datetime.datetime.combine(isoweek.Week(*week).monday(), datetime.time.min)
        time_max = datetime.datetime.combine(isoweek.Week(*week).sunday(), datetime.time.max)
        res[week] = np.random.rand(n) * (time_max-time_min) + time_min
    return res

def uniform_locations_by_county(counties, n=500):
    res = OrderedDict()
    for (county_id, county) in counties.items():
        tp = county["testpoints"]
        if n == len(tp):
            res[county_id] = tp
        else:
            idx = np.random.choice(tp.shape[0], n, replace=n>len(tp))
            res[county_id] = tp[idx]
    return res


def sample_time_and_space(data, times_by_week, locations_by_county):
    n_total = data.sum().sum()
    t_all = np.empty((n_total,), dtype=object)
    x_all = np.empty((n_total,2))
    
    i = 0
    for (county_id, series) in data.iteritems():
        for (week, n) in series.iteritems():
            # draw n random times
            times = times_by_week[week]
            idx = np.random.choice(len(times), n)
            t_all[i:i+n] = times[idx]
            
            # draw n random locations
            locs = locations_by_county[county_id]
            idx = np.random.choice(locs.shape[0], n)
            x_all[i:i+n,:] = locs[idx,:]
            
            i += n
    
    return t_all, x_all

def gaussian_bf(dx, σ):
    σ = np.float32(σ)
    res = tt.zeros_like(dx)
    idx = (abs(dx) < np.float32(5)*σ)#.nonzero()
    return tt.set_subtensor(res[idx], tt.exp(np.float32(-0.5/(σ**2))*(dx[idx])**2)/np.float32(np.sqrt(2*np.pi*σ**2)))

def gaussian_gram(σ):
    return np.array([[np.power(2*np.pi*(a**2 + b**2), -0.5) for b in σ] for a in σ])

def bspline_bfs(x, knots, P):
    knots = knots.astype(np.float32)
    idx = ((x>=knots[0])&(x<knots[-1]))#.nonzero()
    xx = x[idx]
    
    N = {}
    for p in range(P+1):
        for i in range(len(knots)-1-p):
            if p==0:
                N[(i,p)] = tt.where((knots[i]<=xx)*(xx<knots[i+1]), 1.0, 0.0)
            else:
                N[(i,p)] =  (xx-knots[i])/(knots[i+p]-knots[i])*N[(i,p-1)] + \
                            (knots[i+p+1]-xx)/(knots[i+p+1]-knots[i+1])*N[(i+1,p-1)]
                            
    highest_level = []
    for i in range(len(knots)-1-P):
        res = tt.zeros_like(x)
        highest_level.append(tt.set_subtensor(res[idx], N[(i,P)]))
    return highest_level

def jacobian_sq(latitude, R = 6365.902):
    """
        jacobian_sq(latitude)
        
    Computes the "square root" (Cholesky factor) of the Jacobian of the cartesian projection from polar coordinates (in degrees longitude, latitude) onto cartesian coordinates (in km east/west, north/south) at a given latitude (the projection's Jacobian is invariante wrt. longitude).
    """
    return R*(np.pi/180.0)*(abs(tt.cos(tt.deg2rad(latitude)))*np.array([[1.0, 0.0], [0.0, 0.0]])+np.array([[0.0, 0.0],[0.0, 1.0]]))


def build_ia_bfs(temporal_bfs, spatial_bfs):
    x1 = tt.fmatrix("x1")
    t1 = tt.fvector("t1")
    # M = tt.fmatrix("M")
    x2 = tt.fmatrix("x2")
    t2 = tt.fvector("t2")

    lat = x1[:,1].mean()
    M = jacobian_sq(lat)**2
    
    # (x1,t1) are the to-be-predicted points, (x2,t2) the historic cases
    
    # spatial distance btw. each points in x1 and x2 with gramian M
    dx = tt.sqrt((x1.dot(M)*x1).sum(axis=1).reshape((-1,1)) + (x2.dot(M)*x2).sum(axis=1).reshape((1,-1)) - 2*x1.dot(M).dot(x2.T))
    
    # temporal distance btw. each times in t1 and t2
    dt = t1.reshape((-1,1)) - t2.reshape((1,-1))
    
    ft = tt.stack(temporal_bfs(dt.reshape((-1,))),axis=0)
    fx = tt.stack(spatial_bfs(dx.reshape((-1,))),axis=0)
    
    # aggregate contributions of all cases
    contrib = ft.dot(fx.T).reshape((-1,))/tt.cast(x1.shape[0], "float32")
    

                    
    return theano.function([t1,x1,t2,x2], contrib, allow_input_downcast=True)

class IAEffectSampler(object):
    def __init__(self, data, times_by_week, locations_by_county, temporal_bfs, spatial_bfs, num_tps = 10, time_horizon=5, verbose=True):
        self.ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs)
        self.times_by_week = times_by_week
        self.locations_by_county = locations_by_county
        self._to_timestamp = np.frompyfunc(datetime.datetime.timestamp, 1, 1)
        self.data = data
        self.num_tps = num_tps
        self.time_horizon = time_horizon
        self.num_features = len(temporal_bfs(tt.fmatrix("tmp")))*len(spatial_bfs(tt.fmatrix("tmp")))
        self.verbose = verbose
        
    def __call__(self, weeks, counties):
        res = np.zeros((len(weeks), len(counties), self.num_features), dtype=np.float32)
        for i,week in enumerate(weeks):
            for j,county in enumerate(counties):
                idx = ((isoweek.Week(*week)-self.time_horizon) <= self.data.index)*(self.data.index < week)
                # print("sampling week {} for county {} using data in range {}".format(week, county, idx))
                t_data, x_data = sample_time_and_space(self.data.iloc[idx], self.times_by_week, self.locations_by_county)
                t_pred, x_pred = sample_time_and_space(pd.DataFrame(self.num_tps, index=[week], columns=[county]), self.times_by_week, self.locations_by_county)
                res[i,j,:] = self.ia_bfs(self._to_timestamp(t_pred), x_pred, self._to_timestamp(t_data), x_data)
            frac = (i+1)/len(weeks)
            if self.verbose:
                print("⎹" + "█"*int(np.floor(frac*100)) + " ░▒▓█"[int(((frac*100)%1)*5)] + " "*int(np.ceil((1-frac)*100)) + "⎸ ({:.3}%)".format(100*frac), end="\r", flush=True)
        return res


Run shared_utils.py - changed 

In [23]:
yearweek_regex = re.compile(r"([0-9]+)-KW([0-9]+)")

def _parse_yearweek(yearweek):
    """Utility function to convert internal string representations of calender weeks into datetime objects. Uses strings of format `<year>-KW<week>`. Weeks are 1-based."""
    year,week = yearweek_regex.search(yearweek).groups()
    return isoweek.Week(int(year), int(week)) #datetime.combine(isoweek.Week(int(year), int(week)).wednesday(),time(0))
parse_yearweek = np.frompyfunc(_parse_yearweek, 1, 1)

def load_data(disease, prediction_region, counties, separator=","): #changed seperator
    """
        load_data(disease, prediction_region, counties, separator)

    Utility to load the data for a given disease and prediction_region. Only counties are selected that lie in the prediction_region.

    Arguments:
    ==========
        disease:            name of the disease
        prediction_region:  name of the region for which to make a prediction
        counties:           dictionary containing county information
        separator:          string delimiter
        """
    # load raw data from csv file into a pandas dataframe
    data = pd.read_csv("../data/diseases/{}.csv".format(disease),sep=separator,encoding='iso-8859-1',index_col=0)
    # exclude data reported for unknown counties
    if "99999" in data.columns:
        data.drop("99999", inplace=True, axis=1)
    # exclude data for regions outside of region of interest
    data = data.loc[:, list(filter(lambda cid: prediction_region in counties[cid]["region"], data.columns))]
    data.index = parse_yearweek(data.index)
    return data

def split_data(
            data,
            train_start  = parse_yearweek("2011-KW01"),
            test_start = parse_yearweek("2016-KW01"),
            post_test = parse_yearweek("2018-KW01")
        ):
        """
            split_data(data,data_start,train_start,test_start)

        Utility function that splits the dataset into training and testing data as well as the corresponding target values.

        Returns:
        ========
            data_train:     training data (from beginning of records to end of training phase)
            target_train:   target values for training data
            data_test:      testing data (from beginning of records to end of testing phase = end of records)
            target_test:    target values for testing data
        """

        target_train    = data.loc[(train_start <= data.index)  & (data.index < test_start)]
        target_test     = data.loc[(test_start <= data.index) & (data.index < post_test)]

        data_train      = data.loc[data.index < test_start]
        data_test       = data

        return data_train, target_train, data_test, target_test

def quantile_negbin(qs, mean, dispersion=0):
    """ For `n` values in `qs` and `m` values in `mean`, computes the m by n matrix of `qs` quantiles for distributions with means `mean`.
    If dispersion is set to 0, a Poisson distribution is assumed, otherwise a Negative Binomial with corresponding dispersion parameter is used."""
    qs = np.array(qs).ravel()
    mean = np.array(mean).ravel()
    res = np.empty((len(mean), len(qs)))
    for i,mu in enumerate(mean):
        if dispersion == 0:
            # plot Poisson quantiles
            dist = scipy.stats.poisson(mu)
        else:
            # k = y
            n = 1/dispersion
            p = 1/(1+dispersion*mu)
            dist = scipy.stats.nbinom(n,p)
        for j,q in enumerate(qs):
            res[i,j] = dist.ppf(q)
    return res


def deviance_negbin(y, μ, α, saturated="NegativeBinomial"):
    if saturated=="NegativeBinomial":
        logp_sat = tt.where(y==0, np.zeros_like(y,dtype=np.float32), pm.NegativeBinomial.dist(mu=y, alpha=α).logp(y))
    elif saturated=="Poisson":
        logp_sat = tt.where(y==0, np.zeros_like(y,dtype=np.float32), pm.Poisson.dist(mu=y).logp(y))
    else:
        raise NotImplementedError()
    logp_mod = pm.NegativeBinomial.dist(mu=μ, alpha=α).logp(y)
    return (2*(logp_sat - logp_mod)).eval()

def dss(y, μ, var):
    return np.log(var) + (y-μ)**2/var

def pit_negbin(y, μ, α):
    return scipy.stats.nbinom.cdf(y, α, μ/(α + μ))


def make_axes_stack(ax, num, xoff, yoff, fig=None, down=True, sharex=False, sharey=False):
    if isinstance(ax, plt.Axes):
        bbox = ax.get_position()
        if fig is None:
            fig = ax.figure
        ax.set_zorder(num if down else 0)
        axes = [ax]
    elif isinstance(ax, SubplotSpec):
        if fig is None:
            fig = plt.gcf()
        bbox = ax.get_position(fig)
        axes = [fig.add_axes(bbox, zorder = num if down else 0)]
    else:
        if fig is None:
            fig = plt.gcf()
        bbox = transforms.Bbox.from_bounds(ax[0], ax[1], ax[2]-ax[0], ax[3]-ax[1])
        axes = [fig.add_axes(bbox, zorder=num if down else 0)]

    for i in range(1,num):
        bbox = bbox.translated(xoff, yoff)
        ax = fig.add_axes(bbox, zorder=num-i if down else i, sharex=axes[0] if sharex else None, sharey=axes[0] if sharey else None)
        for tick in ax.get_xticklabels() + ax.get_yticklabels():
            tick.set_visible(False)
        axes.append(ax)
    return axes



def set_file_permissions(filename, uid, gid, permissions=0o660):
    os.chmod(filename, permissions)
    os.chown(filename, uid, gid)


def load_model(disease, use_age, use_eastwest):
    filename_model = "../data/mcmc_samples_backup/model_{}_{}_{}.pkl".format(disease, use_age, use_eastwest)

    with open(filename_model,"rb") as f:
        model = pkl.load(f)
    return model

def load_trace(disease, use_age, use_eastwest):
    filename_params = "../data/mcmc_samples_backup/parameters_{}_{}_{}".format(disease, use_age, use_eastwest)

    model = load_model(disease, use_age, use_eastwest)
    with model:
        trace = pm.load_trace(filename_params)
    
    del model
    return trace

def load_pred(disease, use_age, use_eastwest):
    # Load our prediction samples
    filename_pred = "../data/mcmc_samples_backup/predictions_{}_{}_{}.pkl".format(disease, use_age, use_eastwest)
    with open(filename_pred,"rb") as f:
        res = pkl.load(f)
    return res


Run BaseModel.py

In [6]:
class SpatioTemporalFeature(object):
    def __init__(self):
        self._call_ = np.frompyfunc(self.call, 2, 1)

    def __call__(self, times, locations):
        return self._call_(np.asarray(times).reshape((-1,1)), np.asarray(locations).reshape((1,-1))).astype(np.float32)

class SpatioTemporalYearlyDemographicsFeature(SpatioTemporalFeature):
    def __init__(self, county_dict, group, scale=1.0):
        self.dict = {
            (year, county): val*scale
            for county,values in county_dict.items()
            for (g, year),val in values["demographics"].items()
            if g == group
        }
        super().__init__()

    def call(self, yearweek,county):
        return self.dict.get((yearweek[0],county))

class SpatialEastWestFeature(SpatioTemporalFeature):
    def __init__(self, county_dict):
        self.dict = {
            county: 1.0 if "east" in values["region"] else (0.5 if "berlin" in values["region"] else 0.0)
            for county,values in county_dict.items()
        }
        super().__init__()

    def call(self, yearweek,county):
        return self.dict.get(county)

class TemporalFourierFeature(SpatioTemporalFeature):
    def __init__(self, i, t0, scale):
        self.t0 = t0
        self.scale = scale
        self.τ = (i//2 + 1)*2*np.pi
        self.fun = np.sin if (i%2)==0 else np.cos
        super().__init__()

    def call(self, t, x):
        return self.fun((t-self.t0)/self.scale*self.τ)

class TemporalSigmoidFeature(SpatioTemporalFeature):
    def __init__(self, t0, scale):
        self.t0 = t0
        self.scale = scale
        super().__init__()

    def call(self, t, x):
        return sp.special.expit((t-self.t0)/self.scale)


class IAEffectLoader(object):
    generates_stats = False
    def __init__(self, var, filenames, weeks, counties):
        self.vars = [var]
        self.samples = []
        for filename in filenames:
            try:
                with open(filename, "rb") as f:
                    tmp=pkl.load(f)
            except FileNotFoundError:
                print("Warning: File {} not found!".format(filename))
                pass
            except Exception as e:
                print(e)
            else:
                m = tmp["ia_effects"]
                ws = list(tmp["predicted week"])
                cs = list(tmp["predicted county"])
                w_idx = np.array([ws.index(w) for w in weeks]).reshape((-1,1))
                c_idx = np.array([cs.index(c) for c in counties])
                self.samples.append(np.moveaxis(m[w_idx,c_idx,:], -1, 0).reshape((m.shape[-1], -1)).T)

    def step(self, point):
        new = point.copy()
        # res = new[self.vars[0].name]
        new_res = self.samples[np.random.choice(len(self.samples))]
        new[self.vars[0].name] = new_res
        return new

    def stop_tuning(*args):
        pass

    @property
    def vars_shape_dtype(self):
        shape_dtypes = {}
        for var in self.vars:
            dtype = np.dtype(var.dtype)
            shape = var.dshape
            shape_dtypes[var.name] = (shape, dtype)
        return shape_dtypes

class BaseModel(object):
    """
    Model for disease prediction.

    The model has 4 types of features (predictor variables):
    * temporal (functions of time)
    * spatial (functions of space, i.e. longitude, latitude)
    * county_specific (functions of time and space, i.e. longitude, latitude)
    * interaction effects (functions of distance in time and space relative to each datapoint)
    """

    def __init__(self, trange, counties, ia_effect_filenames, num_ia=16, model=None, include_ia=True, include_eastwest=True, include_demographics=True, include_temporal=True, orthogonalize=False):
        self.county_info = counties
        self.ia_effect_filenames = ia_effect_filenames
        self.num_ia = num_ia if include_ia else 0
        self.include_ia = include_ia
        self.include_eastwest = include_eastwest
        self.include_demographics = include_demographics
        self.include_temporal = include_temporal
        self.trange = trange

        first_year=self.trange[0][0]
        last_year=self.trange[1][0]
        self.features = {
                "temporal_seasonal": {"temporal_fourier_{}".format(i): TemporalFourierFeature(i, isoweek.Week(first_year,1), 52.1775) for i in range(4)} if self.include_temporal else {},
                "temporal_trend": {"temporal_sigmoid_{}".format(i): TemporalSigmoidFeature(isoweek.Week(i,1), 2.0) for i in range(first_year,last_year+1)} if self.include_temporal else {},
                "spatiotemporal": {"demographic_{}".format(group): SpatioTemporalYearlyDemographicsFeature(self.county_info, group) for group in ["[0-5)", "[5-20)", "[20-65)"]} if self.include_demographics else {},
                "spatial": {"eastwest": SpatialEastWestFeature(self.county_info)} if self.include_eastwest else {},
                "exposure": {"exposure": SpatioTemporalYearlyDemographicsFeature(self.county_info, "total", 1.0/100000)}
            }
        
        self.Q = np.eye(16, dtype=np.float32)
        if orthogonalize:
            # transformation to orthogonalize IA features
            T = np.linalg.inv(np.linalg.cholesky(gaussian_gram([6.25,12.5,25.0,50.0]))).T
            for i in range(4):
                self.Q[i*4:(i+1)*4, i*4:(i+1)*4] = T


    def evaluate_features(self, weeks, counties):
        all_features = {}
        for group_name,features in self.features.items():
            group_features = {}
            for feature_name,feature in features.items():
                feature_matrix = feature(weeks, counties)
                group_features[feature_name] = pd.DataFrame(feature_matrix[:,:], index=weeks, columns=counties).stack()
            all_features[group_name] = pd.DataFrame([], index=pd.MultiIndex.from_product([weeks,counties]), columns=[]) if len(group_features)==0 else pd.DataFrame(group_features)
        return all_features

    def init_model(self, target):
        weeks,counties = target.index, target.columns

        # extract features
        features = self.evaluate_features(weeks, counties)
        Y_obs = target.stack().values.astype(np.float32)
        T_S = features["temporal_seasonal"].values.astype(np.float32)
        T_T = features["temporal_trend"].values.astype(np.float32)
        TS = features["spatiotemporal"].values.astype(np.float32)
        S = features["spatial"].values.astype(np.float32)

        log_exposure = np.log(features["exposure"].values.astype(np.float32).ravel())

        # extract dimensions
        num_obs = np.prod(target.shape)
        num_t_s = T_S.shape[1]
        num_t_t = T_T.shape[1]
        num_ts = TS.shape[1]
        num_s = S.shape[1]


        with pm.Model() as self.model:
            # interaction effects are generated externally -> flat prior
            IA    = pm.Flat("IA", testval=np.ones((num_obs, self.num_ia)),shape=(num_obs, self.num_ia))

            # priors
            #δ = 1/√α
            δ     = pm.HalfCauchy("δ", 10, testval=1.0)
            α     = pm.Deterministic("α", np.float32(1.0)/δ)
            W_ia  = pm.Normal("W_ia", mu=0, sd=10, testval=np.zeros(self.num_ia), shape=self.num_ia)
            W_t_s = pm.Normal("W_t_s", mu=0, sd=10, testval=np.zeros(num_t_s), shape=num_t_s)
            W_t_t = pm.Normal("W_t_t", mu=0, sd=10, testval=np.zeros(num_t_t), shape=num_t_t)
            W_ts  = pm.Normal("W_ts", mu=0, sd=10, testval=np.zeros(num_ts), shape=num_ts)
            W_s   = pm.Normal("W_s", mu=0, sd=10, testval=np.zeros(num_s), shape=num_s)
            self.param_names = ["δ", "W_ia", "W_t_s", "W_t_t", "W_ts", "W_s"]
            self.params = [δ, W_ia, W_t_s, W_t_t, W_ts, W_s]

            # calculate interaction effect
            IA_ef = tt.dot(tt.dot(IA, self.Q), W_ia)

            # calculate mean rates
            μ = pm.Deterministic("μ", 
                # (1.0+tt.exp(IA_ef))*
                tt.exp(IA_ef + tt.dot(T_S, W_t_s) + tt.dot(T_T, W_t_t) + tt.dot(TS, W_ts) + tt.dot(S, W_s) + log_exposure)
            )

            # constrain to observations
            pm.NegativeBinomial("Y", mu=μ, alpha=α, observed=Y_obs)


    def sample_parameters(self, target, n_init=100, samples=1000, chains=None, cores=8, init="advi", target_accept=0.8, max_treedepth=10, **kwargs):
        """
            sample_parameters(target, samples=1000, cores=8, init="auto", **kwargs)

        Samples from the posterior parameter distribution, given a training dataset.
        The basis functions are designed to be causal, i.e. only data points strictly predating the predicted time points are used (this implies "one-step-ahead"-predictions).
        """
        # model = self.model(target)

        self.init_model(target)

        if chains is None:
            chains = max(2,cores)
            

        with self.model:
            # run!
            ia_effect_loader = IAEffectLoader(self.model.IA, self.ia_effect_filenames, target.index, target.columns)
            nuts = pm.step_methods.NUTS(vars=self.params, target_accept=target_accept, max_treedepth=max_treedepth)
            steps = (([ia_effect_loader] if self.include_ia else [] ) + [nuts] )
            trace = pm.sample(samples, steps, chains=chains, cores=cores, compute_convergence_checks=False, **kwargs)
            # trace = pm.sample(0, steps, tune=samples+tune, discard_tuned_samples=False, chains=chains, cores=cores, compute_convergence_checks=False, **kwargs)
            # trace = trace[tune:]
        return trace

    def sample_predictions(self, target_weeks, target_counties, parameters, init="auto"):
        # extract features
        features = self.evaluate_features(target_weeks, target_counties)

        T_S = features["temporal_seasonal"].values
        T_T = features["temporal_trend"].values
        TS = features["spatiotemporal"].values
        S = features["spatial"].values
        log_exposure = np.log(features["exposure"].values.ravel())

        # extract coefficient samples
        α = parameters["α"]
        W_ia = parameters["W_ia"]
        W_t_s = parameters["W_t_s"]
        W_t_t = parameters["W_t_t"]
        W_ts = parameters["W_ts"]
        W_s = parameters["W_s"]
        

        ia_l = IAEffectLoader(None, self.ia_effect_filenames, target_weeks, target_counties)

        num_predictions = len(target_weeks)*len(target_counties)
        num_parameter_samples = α.size
        y = np.zeros((num_parameter_samples, num_predictions), dtype=int)
        μ = np.zeros((num_parameter_samples, num_predictions), dtype=np.float32)

        for i in range(num_parameter_samples):
            IA_ef = np.dot(np.dot(ia_l.samples[np.random.choice(len(ia_l.samples))], self.Q), W_ia[i])
            # μ[i,:] = (1.0+np.exp(IA_ef))*np.exp(np.dot(T_S, W_t_s[i]) + np.dot(T_T, W_t_t[i]) + np.dot(TS, W_ts[i]) + np.dot(S, W_s[i]) + log_exposure)
            μ[i,:] = np.exp(IA_ef + np.dot(T_S, W_t_s[i]) + np.dot(T_T, W_t_t[i]) + np.dot(TS, W_ts[i]) + np.dot(S, W_s[i]) + log_exposure)
            y[i,:] = pm.NegativeBinomial.dist(mu=μ[i,:], alpha=α[i]).random()

        return {"y": y, "μ": μ, "α": α}


Run sample_ia_effects.py - changed

In [9]:
def run_sampling(disease, num_samples=100):
    print(f"\n▶ Starting sampling for: {disease}")
    
    for num_sample in range(num_samples):
        filename = f"../data/ia_effect_samples/{disease}_{num_sample}.pkl"
        print(f"⏳ Sample {num_sample} - creating {filename}")

        with open('../data/counties/counties.pkl', "rb") as f:
            counties = pkl.load(f)

        prediction_region = "bavaria" if disease == "borreliosis" else "germany"
        parameters = OrderedDict()

        # Load data
        data = load_data(disease, prediction_region, counties)

        times = uniform_times_by_week(data.index)
        locs = uniform_locations_by_county(counties)
        temporal_bfs = lambda x: bspline_bfs(x, np.array([0, 0, 1, 2, 3, 4, 5]) * 7 * 24 * 3600.0, 2)
        spatial_bfs = lambda x: [gaussian_bf(x, σ) for σ in [6.25, 12.5, 25.0, 50.0]]

        samp = IAEffectSampler(data, times, locs, temporal_bfs, spatial_bfs, num_tps=10, time_horizon=5)
        res = samp(data.index, data.columns)

        results = {
            "ia_effects": res,
            "predicted week": data.index,
            "predicted county": data.columns
        }

        with open(filename, "wb") as file:
            pkl.dump(results, file)

        print(f"✅ Finished sample {num_sample} for {disease}")

#diseases    = ["campylobacter", "rotavirus", "borreliosis"]

#for disease in diseases:
#    run_sampling(disease, num_samples=100)

run_sampling("borreliosis")


▶ Starting sampling for: borreliosis
⏳ Sample 0 - creating ../data/ia_effect_samples/borreliosis_0.pkl


c:\Users\Ellaine2\miniconda3\envs\env_bstim\lib\site-packages\theano\tensor\subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
c:\Users\Ellaine2\miniconda3\envs\env_bstim\lib\site-packages\theano\tensor\subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


✅ Finished sample 0 for borreliosis██████████████████████████████████████████████████████████████████ ⎸ (1e+02%)
⏳ Sample 1 - creating ../data/ia_effect_samples/borreliosis_1.pkl
✅ Finished sample 1 for borreliosis██████████████████████████████████████████████████████████████████ ⎸ (1e+02%)
⏳ Sample 2 - creating ../data/ia_effect_samples/borreliosis_2.pkl
✅ Finished sample 2 for borreliosis██████████████████████████████████████████████████████████████████ ⎸ (1e+02%)
⏳ Sample 3 - creating ../data/ia_effect_samples/borreliosis_3.pkl
✅ Finished sample 3 for borreliosis██████████████████████████████████████████████████████████████████ ⎸ (1e+02%)
⏳ Sample 4 - creating ../data/ia_effect_samples/borreliosis_4.pkl
✅ Finished sample 4 for borreliosis██████████████████████████████████████████████████████████████████ ⎸ (1e+02%)
⏳ Sample 5 - creating ../data/ia_effect_samples/borreliosis_5.pkl
✅ Finished sample 5 for borreliosis██████████████████████████████████████████████████████████████████ ⎸ (

Run Comfig changed

In [17]:
diseases = ["borreliosis"]
prediction_regions = ["bavaria"]

combinations_age_eastwest = [(True,False),(False,False)]
combinations = list(it.product(range(len(combinations_age_eastwest)), diseases))

print(combinations)

[(0, 'borreliosis'), (1, 'borreliosis')]


In [20]:
num_samples = 250
num_chains = 4
num_cores = 1

for i in range(len(combinations)):

    model_complexity, disease = combinations[i]
    use_age, use_eastwest     = combinations_age_eastwest[model_complexity]

    prediction_region = "bavaria"

    filename_params = "../data/mcmc_samples_backup/parameters_{}_{}_{}".format(disease, use_age, use_eastwest)
    filename_pred = "../data/mcmc_samples_backup/predictions_{}_{}_{}.pkl".format(disease, use_age, use_eastwest)
    filename_model = "../data/mcmc_samples_backup/model_{}_{}_{}.pkl".format(disease, use_age, use_eastwest)

    # Load data
    with open('../data/counties/counties.pkl',"rb") as f:
        county_info = pkl.load(f)

    data_posterior = load_data(disease, prediction_region, county_info)
    data_posterior_train, target_posterior_train, data_posterior_test, target_posterior_test = split_data(data_posterior)

    # Remove KW53 if present
    # target_posterior_train = target_posterior_train[~(target_posterior_train.index == parse_yearweek("2015-KW53"))]

    print(target_posterior_train.index[-1])

    tspan = (target_posterior_train.index[0],target_posterior_train.index[-1])

    print("training for {} in {} with model complexity {} from {} to {}\nWill create files {}, {} and {}".format(disease, prediction_region, model_complexity,*tspan, filename_params, filename_pred, filename_model))

    model = BaseModel(tspan, county_info, ["../data//ia_effect_samples/{}_{}.pkl".format(disease, i) for i in range(100)], include_eastwest=use_eastwest, include_demographics=use_age)

    print("Sampling parameters on the training set.")
    trace = model.sample_parameters(target_posterior_train, samples=num_samples, tune=100, target_accept=0.95, max_treedepth=15, chains=num_chains, cores=num_cores)

    with open(filename_model, "wb") as f:
        pkl.dump(model.model, f)

    with model.model:
        pm.save_trace(trace, filename_params, overwrite=True)

    print("Sampling predictions on the testing set.")
    pred = model.sample_predictions(target_posterior_test.index, target_posterior_test.columns, trace)
    with open(filename_pred, 'wb') as f:
        pkl.dump(pred, f)

2015W53
training for borreliosis in bavaria with model complexity 0 from 2013W01 to 2015W53
Will create files ../data/mcmc_samples_backup/parameters_borreliosis_True_False, ../data/mcmc_samples_backup/predictions_borreliosis_True_False.pkl and ../data/mcmc_samples_backup/model_borreliosis_True_False.pkl
Sampling parameters on the training set.


Only 250 samples in chain.
Sequential sampling (4 chains in 1 job)
CompoundStep
>IAEffectLoader: [IA]
>NUTS: [W_s, W_ts, W_t_t, W_t_s, W_ia, δ]
100%|██████████| 350/350 [6:55:57<00:00, 71.31s/it]    


Sampling predictions on the testing set.


INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock


2015W53
training for borreliosis in bavaria with model complexity 1 from 2013W01 to 2015W53
Will create files ../data/mcmc_samples_backup/parameters_borreliosis_False_False, ../data/mcmc_samples_backup/predictions_borreliosis_False_False.pkl and ../data/mcmc_samples_backup/model_borreliosis_False_False.pkl
Sampling parameters on the training set.


Only 250 samples in chain.
Sequential sampling (4 chains in 1 job)
CompoundStep
>IAEffectLoader: [IA]
>NUTS: [W_s, W_ts, W_t_t, W_t_s, W_ia, δ]
100%|██████████| 350/350 [13:40<00:00,  2.34s/it]


Sampling predictions on the testing set.


INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock


sampling_predictions - changed

In [24]:
age_eastwest_by_name = dict(zip(["A", "B"], combinations_age_eastwest))

with open('../data/counties/counties.pkl', "rb") as f:
    county_info = pkl.load(f)

best_model = {}

for disease in diseases:
    print("Evaluating model for {}...".format(disease))
    prediction_region = "bavaria"

    data = load_data(disease, prediction_region, county_info)
    data_train, target_train, data_test, target_test = split_data(data)
    tspan = (target_train.index[0], target_train.index[-1])
    waics = {}

    for (name, (use_age, use_eastwest)) in age_eastwest_by_name.items():
        # Redundant but explicit: force use_eastwest to False
        use_eastwest = False

        trace = load_trace(disease, use_age, use_eastwest)

        model = BaseModel(
            tspan,
            county_info,
            ["../data/ia_effect_samples/{}_{}.pkl".format(disease, i) for i in range(100)],
            include_eastwest=use_eastwest,
            include_demographics=use_age
        )

        filename_pred = "../data/mcmc_samples_backup/predictions_{}_{}_{}.pkl".format(
            disease, use_age, use_eastwest
        )
        print("Sampling predictions on the testing set.")
        pred = model.sample_predictions(target_test.index, target_test.columns, trace)

        with open(filename_pred, 'wb') as f:
            pkl.dump(pred, f)

        del trace
        del model

Evaluating model for borreliosis...
Sampling predictions on the testing set.


INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilel

Sampling predictions on the testing set.


INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Ellaine2\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_25_Model_80_Stepping_0_AuthenticAMD-3.7.1-64\lock_dir\lock
INFO (theano.gof.compilel